In [5]:
import json
from collections import defaultdict
import glob
import os


# List of JSON files you want to combine
json_files = [
    "C:/Users/w065pxg/Desktop/ML/Tanvi Banerjee 2024 MI/Final Project/StudentResearch/final/Datacollection/Lonleytogether.json",
    "C:/Users/w065pxg/Desktop/ML/Tanvi Banerjee 2024 MI/Final Project/StudentResearch/final/Datacollection/redditloneliness.json",
    "C:/Users/w065pxg/Desktop/ML/Tanvi Banerjee 2024 MI/Final Project/StudentResearch/final/Datacollection/rForeverAlone.json",
    "C:/Users/w065pxg/Desktop/ML/Tanvi Banerjee 2024 MI/Final Project/StudentResearch/final/Datacollection/rlonely.json"
]


combined_data = {
    "meta": {},
    "flair": defaultdict(dict)
}

# For logging issues
conflicts = {
    "meta_duplicates": [],
    "flair_duplicates": []
}

for file in json_files:
    if not os.path.exists(file):
        print(f"❌ File not found: {file}")
        continue

    try:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except json.JSONDecodeError:
        print(f"❌ Failed to decode JSON: {file}")
        continue

    # ✅ Merge meta safely
    for post_id, post_meta in data.get("meta", {}).items():
        if post_id in combined_data["meta"]:
            conflicts["meta_duplicates"].append((file, post_id))
        combined_data["meta"][post_id] = post_meta

    # ✅ Merge flair safely
    for flair, posts in data.get("flair", {}).items():
        for post_id, post_content in posts.items():
            if post_id in combined_data["flair"][flair]:
                conflicts["flair_duplicates"].append((file, flair, post_id))
            combined_data["flair"][flair][post_id] = post_content

# Convert defaultdict to dict
combined_data["flair"] = dict(combined_data["flair"])

# Save combined file
output_file = "redditdata_combined.json"
with open(output_file, "w", encoding="utf-8") as out_file:
    json.dump(combined_data, out_file, indent=2)

print(f"\n✅ Combined data saved to: {output_file}")

# Report conflicts
if conflicts["meta_duplicates"] or conflicts["flair_duplicates"]:
    print("\n⚠️ Merge conflicts detected:")
    if conflicts["meta_duplicates"]:
        print("  - Meta Duplicates:")
        for file, pid in conflicts["meta_duplicates"]:
            print(f"    {file} → post ID: {pid}")
    if conflicts["flair_duplicates"]:
        print("  - Flair Duplicates:")
        for file, flair, pid in conflicts["flair_duplicates"]:
            print(f"    {file} → flair: {flair}, post ID: {pid}")
else:
    print("✅ No merge conflicts found.")


✅ Combined data saved to: redditdata_combined.json
✅ No merge conflicts found.
